<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-11 13:01:01
-------------------
qualified stocks: 86
with latest results: 24
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  99.50 K
Current:  1.43 C
-------------------
Today PnL: -31.03 K (-0.22%)
Current PnL: -22.50 L (-14.74%)
CY Booked + Current PnL: -9.31 L (-6.1%)
-------------------
Total profit:  1.58 L
Total loss:  -24.08 L
-------------------
Total Booked + Current PnL: 17.67 L (14.07%)
Total Booked PnL: 40.17 L (31.98%)
Curr Year Booked PnL: 13.19 L (9.25%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 89.96 L (63.08%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.38%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,101.71,51.0,M-SC,2.20,86860.0,-13917.0,14011.0,0.36,-13.81,16.13,0.09,245.0,-0.99,0.61,14.11,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.05,48.0,X-MC,3.14,88815.0,-1863.0,16768.0,0.34,-2.05,18.88,16.44,101.0,-0.11,0.63,17.71,X40,ATH,PHARMA
42,ITC,452.00,-38.34,45.0,X-LC,0.83,198534.0,-1604.0,22494.0,0.11,-0.80,11.33,10.44,4.0,-0.07,1.40,4.72,X40,NTT,FMCG
34,ICICIGI,2252.93,-13.84,55.0,X-MC,6.60,186214.0,11894.0,23314.0,-1.14,6.82,12.52,20.19,91.0,0.51,1.31,22.97,X40,ATH,INSURANCE
51,NESTLEIND,1377.00,-3.77,59.0,X-LC,8.21,295987.0,30561.0,26224.0,0.16,11.51,8.86,21.40,11.0,1.17,2.09,18.92,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,680.0,-17.39,57.0,X-MC,1.11,222082.0,-5257.0,53322.0,2.38,-2.31,24.01,21.14,106.0,-0.10,1.57,25.18,XY24,NTT,PAINTS
39,INDUSINDBK,1800.0,280.25,69.0,L-MC,7.39,46632.0,-30574.0,55968.0,2.35,-39.60,120.02,32.89,258.0,-0.55,0.33,28.42,XR,NTT,BANKS
76,TITAGARH,1548.0,-7.80,45.0,H-SC,3.95,196764.0,-45413.0,156172.0,1.80,-18.75,79.37,45.74,158.0,-0.29,1.39,27.98,XY24,NTT,ENGINEERING
38,INDIGOPNTS,1408.0,122.10,76.0,M-SC,5.36,145731.0,-28828.0,28855.0,1.72,-16.51,19.80,0.02,221.0,-1.00,1.03,26.76,OX40N,NTT,PAINTS
5,ANGELONE,3033.0,22.45,68.0,X-SC,6.68,216759.0,25753.0,31950.0,1.42,13.48,14.74,30.21,157.0,0.81,1.53,37.18,X40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,240.00,46.93,33.0,H-SC,13.84,133926.0,-45145.0,110877.0,-2.91,-25.21,82.79,36.71,135.0,-0.41,0.95,32.88,XR,NTT,BANKS
26,GREENPANEL,537.00,197.16,37.0,M-SC,5.23,132541.0,-50537.0,129519.0,-2.69,-27.60,97.72,43.14,230.0,-0.39,0.94,24.67,XY24,NTT,MISC
83,VOLTAS,1530.00,-6.67,36.0,X-MC,4.02,195885.0,4143.0,33614.0,-2.60,2.16,17.16,19.69,99.0,0.12,1.38,9.11,XY25,NTT,AC
84,WHIRLPOOL,2270.00,-44.33,39.0,M-SC,8.43,115515.0,717.0,93325.0,-2.30,0.62,80.79,81.92,223.0,0.01,0.82,36.75,XR,NTT,DURABLES
25,GILLETTE,11206.78,-19.16,16.0,X-MC,5.55,248460.0,-5436.0,87731.0,-2.29,-2.14,35.31,32.42,105.0,-0.06,1.75,12.63,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-44.33,39.0,M-SC,8.43,115515.0,717.0,93325.0,-2.3,0.62,80.79,81.92,223.0,0.01,0.82,36.75,XR,NTT,DURABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.71,51.0,M-SC,2.20,86860.0,-13917.0,14011.0,0.36,-13.81,16.13,0.09,245.0,-0.99,0.61,14.11,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.0,122.10,76.0,M-SC,5.36,145731.0,-28828.0,28855.0,1.72,-16.51,19.80,0.02,221.0,-1.00,1.03,26.76,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-68.16,40.0,H-SC,2.27,219528.0,-50139.0,86472.0,-0.77,-18.59,39.39,13.47,138.0,-0.58,1.55,11.62,XY24,NTT,PAINTS
67,SIS,528.0,2023.32,47.0,H-SC,2.20,85680.0,-25352.0,48958.0,0.37,-22.83,57.14,21.26,156.0,-0.52,0.60,15.76,OX40N,NTT,MISC
43,JCHAC,2282.0,19943.48,63.0,M-SC,1.49,97922.0,-29883.0,29866.0,0.02,-23.38,30.50,-0.01,232.0,-1.00,0.69,10.88,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,57.18,69.0,M-MC,10.48,247796.0,22834.0,144812.0,-0.30,10.15,58.44,74.53,192.0,0.16,1.75,45.68,XY24,BTT,STEEL
32,HINDZINC,730.22,29.16,53.0,M-LC,9.17,210627.0,5551.0,107020.0,-0.08,2.71,50.81,54.89,52.0,0.05,1.49,27.45,X5K,ATH,METALS
84,WHIRLPOOL,2270.00,-44.33,39.0,M-SC,8.43,115515.0,717.0,93325.0,-2.30,0.62,80.79,81.92,223.0,0.01,0.82,36.75,XR,NTT,DURABLES
37,INDIAMART,4810.62,-53.89,48.0,H-SC,5.06,128525.0,5189.0,126443.0,-1.09,4.21,98.38,106.72,119.0,0.04,0.91,28.57,AR,ATH,MISC
62,SAMMAANCAP,326.00,-192.05,53.0,M-SC,35.14,158670.0,9450.0,134727.0,-1.64,6.33,84.91,96.62,208.0,0.07,1.12,71.72,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,29.16,53.0,M-LC,9.17,210627.0,5551.0,107020.0,-0.08,2.71,50.81,54.89,52.0,0.05,1.49,27.45,X5K,ATH,METALS
37,INDIAMART,4810.62,-53.89,48.0,H-SC,5.06,128525.0,5189.0,126443.0,-1.09,4.21,98.38,106.72,119.0,0.04,0.91,28.57,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-44.33,39.0,M-SC,8.43,115515.0,717.0,93325.0,-2.30,0.62,80.79,81.92,223.0,0.01,0.82,36.75,XR,NTT,DURABLES
85,WIPRO,420.00,-15.15,47.0,M-LC,5.68,149315.0,-1630.0,111090.0,0.41,-1.08,74.40,72.51,53.0,-0.01,1.05,4.88,XR,NTT,IT
47,KPIGREEN,731.05,3.26,45.0,H-SC,9.80,123115.0,-2182.0,61114.0,0.88,-1.74,49.64,47.03,141.0,-0.04,0.87,54.15,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GILLETTE,11206.78,-19.16,16.0,X-MC,5.55,248460.0,-5436.0,87731.0,-2.29,-2.14,35.31,32.42,105.0,-0.06,1.75,12.63,X40,ATH,FMCG
74,TATAMOTORS,1065.00,-53.17,24.0,X-LC,36.18,148993.0,-127530.0,237599.0,1.17,-46.12,159.47,39.81,54.0,-0.54,1.05,5.01,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.49,86926.0,-41744.0,87039.0,-0.21,-32.44,100.13,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
45,JSWINFRA,342.00,-24.37,29.0,X-MC,4.43,190012.0,-10335.0,40834.0,0.97,-5.16,21.49,15.23,178.0,-0.25,1.34,25.43,X40N,NTT,REALTY
23,DMART,5391.45,-20.66,30.0,X-LC,6.77,202245.0,-8162.0,67327.0,0.64,-3.88,33.29,28.12,38.0,-0.12,1.43,18.83,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PGHH,17907.65,-32.13,33.0,X-MC,0.67,196425.0,-4395.0,72186.0,-1.42,-2.19,36.75,33.76,80.0,-0.06,1.39,2.89,X40,ATH,FMCG
42,ITC,452.00,-38.34,45.0,X-LC,0.83,198534.0,-1604.0,22494.0,0.11,-0.80,11.33,10.44,4.0,-0.07,1.40,4.72,X40,NTT,FMCG
66,SIEMENS,4671.50,-6.25,36.0,H-LC,1.63,150535.0,-35560.0,83035.0,-1.43,-19.11,55.16,25.51,15.0,-0.43,1.06,10.93,AR,ATH,ELECTRICAL
29,HAVELLS,2069.16,-9.30,36.0,X-MC,3.32,302820.0,-25171.0,131696.0,-1.08,-7.67,43.49,32.48,92.0,-0.19,2.14,2.87,X40,ATH,ELECTRICAL
79,UNITDSPR,1644.00,-6.38,62.0,X-LC,3.60,245375.0,11231.0,37395.0,1.21,4.80,15.24,20.77,86.0,0.30,1.73,12.05,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.49,86926.0,-41744.0,87039.0,-0.21,-32.44,100.13,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
54,QUESS,986.00,-51.52,31.0,X-SC,37.27,47983.0,-17023.0,167950.0,-2.14,-26.19,350.02,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
1,ABB,7934.00,-43.08,31.0,H-LC,5.00,237840.0,-23779.0,142989.0,-0.74,-9.09,60.12,45.57,7.0,-0.17,1.68,1.20,AR,NTT,ELECTRICAL
20,COLPAL,3726.84,-7.40,39.0,X-MC,7.14,216160.0,-47205.0,156521.0,-0.44,-17.92,72.41,41.51,84.0,-0.30,1.53,1.44,XY25,ATH,FMCG
52,PAGEIND,51769.93,-29.12,38.0,X-MC,7.64,160240.0,-3500.0,46838.0,-0.35,-2.14,29.23,26.47,82.0,-0.07,1.13,1.52,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-51.52,31.0,X-SC,37.27,47983.0,-17023.0,167950.0,-2.14,-26.19,350.02,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-40.65,41.0,X-SC,4.03,80840.0,-64320.0,140249.0,0.48,-44.31,173.49,52.31,136.0,-0.46,0.57,8.21,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.49,86926.0,-41744.0,87039.0,-0.21,-32.44,100.13,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.05,48.0,X-MC,3.14,88815.0,-1863.0,16768.0,0.34,-2.05,18.88,16.44,101.0,-0.11,0.63,17.71,X40,ATH,PHARMA
33,HONAUT,58357.33,-24.36,36.0,X-SC,0.92,106545.0,-21393.0,68530.0,-1.39,-16.72,64.32,36.84,143.0,-0.31,0.75,9.08,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,-28.36,52.0,X-LC,9.36,285290.0,-60666.0,127353.0,0.32,-17.54,44.64,19.28,1.0,-0.48,2.01,5.47,X40,ATH,IT
40,INFY,2275.00,-16.73,62.0,X-LC,4.71,324527.0,11639.0,160057.0,0.67,3.72,49.32,54.87,3.0,0.07,2.29,12.31,X40,BTT,IT
42,ITC,452.00,-38.34,45.0,X-LC,0.83,198534.0,-1604.0,22494.0,0.11,-0.80,11.33,10.44,4.0,-0.07,1.40,4.72,X40,NTT,FMCG
82,VBL,671.64,-17.48,50.0,X-LC,5.75,297372.0,-18470.0,131141.0,0.62,-5.85,44.10,35.67,5.0,-0.14,2.10,7.20,X40N,ATH,FMCG
1,ABB,7934.00,-43.08,31.0,H-LC,5.00,237840.0,-23779.0,142989.0,-0.74,-9.09,60.12,45.57,7.0,-0.17,1.68,1.20,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,42.0,L-SC,27.24,76800.0,-36749.0,76800.0,-1.57,-32.36,100.00,35.27,268.0,-0.48,0.54,87.97,XR,NTT,HOTELS
7,ASIANTILES,137.00,7438.89,62.0,L-SC,13.97,82738.0,-11072.0,87686.0,-1.60,-11.80,105.98,81.67,269.0,-0.13,0.58,60.23,XR,NTT,CERAMICS
50,MASFIN,398.61,-19.60,47.0,H-SC,7.92,91185.0,-6795.0,28395.0,-0.34,-6.94,31.14,22.05,152.0,-0.24,0.64,32.54,XR,ATH,FINANCE
47,KPIGREEN,731.05,3.26,45.0,H-SC,9.80,123115.0,-2182.0,61114.0,0.88,-1.74,49.64,47.03,141.0,-0.04,0.87,54.15,MH,ATH,POWER
38,INDIGOPNTS,1408.00,122.10,76.0,M-SC,5.36,145731.0,-28828.0,28855.0,1.72,-16.51,19.80,0.02,221.0,-1.00,1.03,26.76,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VAIBHAVGBL,521.00,67.49,57.0,H-SC,5.62,145371.0,-37404.0,148467.0,-1.70,-20.46,102.13,60.77,125.0,-0.25,1.03,30.92,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7438.89,62.0,L-SC,13.97,82738.0,-11072.0,87686.0,-1.60,-11.80,105.98,81.67,269.0,-0.13,0.58,60.23,XR,NTT,CERAMICS
5,ANGELONE,3033.00,22.45,68.0,X-SC,6.68,216759.0,25753.0,31950.0,1.42,13.48,14.74,30.21,157.0,0.81,1.53,37.18,X40N,NTT,FINANCE
38,INDIGOPNTS,1408.00,122.10,76.0,M-SC,5.36,145731.0,-28828.0,28855.0,1.72,-16.51,19.80,0.02,221.0,-1.00,1.03,26.76,OX40N,NTT,PAINTS
6,ASIANPAINT,3465.66,-7.83,72.0,X-LC,6.15,232505.0,-19263.0,72472.0,-0.31,-7.65,31.17,21.13,27.0,-0.27,1.64,25.06,X40,ATH,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.84
1,25,44.82
2,50,76.19


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.47
LC    30.48
MC    26.02
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.81
X40      19.31
X40N     12.48
XY25     10.22
XR        9.69
AR        7.72
OX40N     6.58
X200      1.79
X5K       1.49
SR        1.01
MH        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.92
X-MC    21.62
X-LC    21.01
M-SC    12.52
X-SC     5.58
H-LC     4.53
M-LC     3.94
H-MC     2.32
M-MC     1.75
L-SC     1.45
L-LC     1.00
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.00,-4.54,38.35
IT,12.61,-17.55,80.47
FINANCE,12.15,-9.90,59.72
MISC,7.70,-19.16,79.27
PAINTS,5.79,-12.62,29.41
ELECTRICAL,5.63,-13.28,53.43
INSURANCE,4.48,0.37,34.75
AUTO,2.72,-49.15,111.40
BANKS,2.70,-40.06,131.34


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3309544.0,21
XR,1284487.0,13
AR,1193296.0,9
X40,955086.0,13
X40N,700563.0,9
OX40N,550277.0,9
XY25,487595.0,7
SR,277156.0,2
X5K,107020.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3442293.0,24
X-MC,1348269.0,15
M-SC,1272404.0,15
X-LC,1061920.0,12
X-SC,557545.0,6
H-LC,296122.0,3
M-LC,265379.0,3
H-MC,258369.0,2
L-SC,254805.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1191818.0      6
           AR         842808.0      5
           XR         810440.0      7
M-SC       XY24       744267.0      6
X-MC       XY24       568076.0      4
X-LC       X40        440133.0      5
X-MC       X40        427914.0      7
X-LC       XY24       308810.0      2
X-SC       X40N       302556.0      4
H-SC       SR         277156.0      2
           OX40N      258957.0      3
X-LC       X40N       235863.0      3
H-LC       AR         226024.0      2
X-MC       XY25       190135.0      2
H-MC       XY24       183811.0      1
X-SC       XY24       167950.0      1
L-SC       XR         164486.0      2
X-MC       X40N       162144.0      2
M-MC       XY24       144812.0      1
M-SC       XR         142503.0      2
           XY25       134727.0      1
           OX40N      126443.0      4
           AR         124464.0      2
M-LC       XR         111090.0      1
           X5K        107020.0      1
L-SC       OX40N       90319.0      1
X-SC       X40         87039.0      1
X-LC       XY25        77114.0      2
H-MC       OX40N       74558.0      1
H-LC       X200        70098.0      1
H-SC       MH          61114.0      1
L-MC       XR          55968.0      1
M-LC       XY25        47269.0      1
L-LC       XY25        38350.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
